In [1]:
# environment
import os
import sys
root_path = os.path.abspath("../")
sys.path.append(root_path)

In [2]:
import pandas as pd

In [3]:
from etls.sheets.pat_pop import PatpopScrub
from etls.sheets.va_cols import VARuleReader, get_occs
from etls.etl_helpers import try_strip
from etls.instructions import instructions

In [4]:
def display_check(result):
    old_col = result[0]
    new_cols = result[1]
    diff = old_col.compare(
        new_cols[0], result_names=(f"old_{old_col.name}", new_cols[0].name)
    )
    if len(new_cols) > 1:
        idx = list(diff.index)
        cols = [old_col] + new_cols
        return pd.DataFrame(cols).T.loc[idx]
    return diff

In [5]:
df = pd.read_csv("../data/interim/ophthafterdxdate.csv")

C:\Users\PANDA15-User\AppData\Local\Temp\ipykernel_27868\1523278632.py:1: DtypeWarning: Columns (6,10,11,28,32,40,42,46,48) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../data/interim/ophthafterdxdate.csv")


# We need to account for .xlsx to .csv conversion when reconciling with our data cleaning summary
- "n/a" is converted to NaN when we convert .xlsx to .csv (already handled; subtract from expected)
- leading and trailing spaces are removed when we convert .xlsx to .csv (the diff between actual and expected should be added)


In [6]:
col_name = "VA_Right_Pressure"

In [7]:
df[col_name] = [try_strip(x) for x in df[col_name]]

In [8]:
col = df[col_name]

In [9]:
ins = instructions["ophthafterdxdate"][col_name]

In [10]:
result = PatpopScrub(col, ins).clean()

In [15]:
rules = get_occs(ins["args"])

In [16]:
rules = pd.DataFrame(rules.groupby(["Value", "Rule (to be defined by CCF team)"])["Occurences"].sum()).reset_index()

In [17]:
def mod_expected(df, col_name, rules, expected):
    rules_merge = pd.merge(rules, df[[col_name]], left_on=["Value"], right_on=[col_name], how="left")
    rules_merge["actual_occurences"] = rules_merge.groupby(col_name)[col_name].transform("size")
    mis_match = rules_merge[rules_merge["Occurences"] != rules_merge["actual_occurences"]]
    dups_dropped = mis_match.drop_duplicates(subset=["Value"])
    nan_count = sum(dups_dropped[dups_dropped["Value"].isnull()]["Occurences"])
    occ_diff = dups_dropped[~dups_dropped["Value"].isnull()]
    act_minus_exp = sum(occ_diff["actual_occurences"]) - sum(occ_diff["Occurences"])

    return expected - nan_count + act_minus_exp
    




In [18]:
rules_merge = pd.merge(rules, df[[col_name]], left_on=["Value"], right_on=[col_name], how="left")

In [25]:
rules_merge

,Value,Rule (to be defined by CCF team),Occurences,VA_Right_Pressure,actual_occurences
0,-,Replace with blank,2,-,2
1,-,Replace with blank,2,-,2
2,--,Replace with blank,10,--,10
3,--,Replace with blank,10,--,10
4,--,Replace with blank,10,--,10
...,...,...,...,...,...
127,x,Replace with blank,9,x,9
128,x,Replace with blank,9,x,9
129,x,Replace with blank,9,x,9
130,x,Replace with blank,9,x,9


In [19]:
rules_merge["actual_occurences"] = rules_merge.groupby(col_name)[col_name].transform("size")

In [41]:
rules_merge

,Value,Rule (to be defined by CCF team),Occurences,VA_Right_Pressure,actual_occurences
0,-,Replace with blank,2,-,2
1,-,Replace with blank,2,-,2
2,--,Replace with blank,10,--,10
3,--,Replace with blank,10,--,10
4,--,Replace with blank,10,--,10
...,...,...,...,...,...
127,x,Replace with blank,9,x,9
128,x,Replace with blank,9,x,9
129,x,Replace with blank,9,x,9
130,x,Replace with blank,9,x,9


In [42]:
rules_merge[rules_merge["Value"].isna()]

,Value,Rule (to be defined by CCF team),Occurences,VA_Right_Pressure,actual_occurences


In [38]:
mis_match = rules_merge[rules_merge["Occurences"] != rules_merge["actual_occurences"]]

In [39]:
mis_match

,Value,Rule (to be defined by CCF team),Occurences,VA_Right_Pressure,actual_occurences


In [27]:
dups_dropped =mis_match.drop_duplicates(subset=["Value"])

In [28]:
dups_dropped

,Value,Rule (to be defined by CCF team),Occurences,VA_Right_Pressure,actual_occurences


In [29]:
nan_count = sum(dups_dropped[dups_dropped["Value"].isnull()]["Occurences"])

In [30]:
nan_count

0

In [31]:
occ_diff = dups_dropped[~dups_dropped["Value"].isnull() & ~dups_dropped['actual_occurences'].isnull()]

In [33]:
cc_diff

NameError: name 'cc_diff' is not defined

In [32]:
occ_diff

,Value,Rule (to be defined by CCF team),Occurences,VA_Right_Pressure,actual_occurences


In [34]:
sum(occ_diff["actual_occurences"])

0

In [35]:
sum(occ_diff["Occurences"])

0

In [36]:
sum(occ_diff["actual_occurences"]) - sum(occ_diff["Occurences"])

0

***

In [ ]:
occ_diff = 

In [46]:
rules_merge["Value"].value_counts()

Value
--              20
x               13
soft            12
UTO             11
X                9
---              8
uto              6
CL               6
dfr              5
unab             4
defe             4
NT               4
Soft             3
sft              3
Pros             2
Dfr              2
Prosthetic       1
nl               1
Unable           1
pros             1
16?              1
STP              1
20?              1
18-1             1
`18              1
21-2             1
stp              1
22, 24           1
BCL in place     1
BCL              1
DEF              1
-                1
err              1
0-10             1
<5               1
DFR              1
NTT              1
Name: count, dtype: int64

In [59]:
str_rule_count = str_rules.rules["Occurences"]


In [61]:
str_rule_count.name = "Occurences"

In [64]:
str_rule_count

0     20
1     13
2     12
3     11
4      9
5      8
6      6
7      5
8      4
9      4
10     4
11     3
12     3
13     3
14     2
15     2
16     1
17     1
18     1
19     1
20     1
21     1
22     1
23     1
24     1
25     1
26     1
27     1
28     1
29     1
30     1
31     1
32     1
33     1
34     1
35     1
36     1
37     1
38     1
39     1
Name: Occurences, dtype: int64

In [65]:
pd.Series(rules_merge["Value"].value_counts().values, name="Occurences")

0     20
1     13
2     12
3     11
4      9
5      8
6      6
7      6
8      5
9      4
10     4
11     4
12     3
13     3
14     2
15     2
16     1
17     1
18     1
19     1
20     1
21     1
22     1
23     1
24     1
25     1
26     1
27     1
28     1
29     1
30     1
31     1
32     1
33     1
34     1
35     1
36     1
Name: Occurences, dtype: int64

In [ ]:
from etls.sheets.va_cols import VARuleReader


rules= VARuleReader()

In [23]:
display_check(result)

,old_VA_Left_Pressure,VA_Left_Pressure
2679,soft,NaN
2680,soft,NaN
2689,X,NaN
5004,--,NaN
6277,Soft,NaN
...,...,...
105684,UTO,NaN
105685,UTO,NaN
105861,---,NaN
107253,defe,NaN


In [19]:
133-129

4

In [13]:
orig

0         NaN
1         NaN
2          19
3          15
4         NaN
         ... 
108369    NaN
108370     17
108371     16
108372    NaN
108373    NaN
Name: VA_Left_Pressure, Length: 108374, dtype: object

In [14]:
ammend

[0         NaN
 1         NaN
 2          19
 3          15
 4         NaN
          ... 
 108369    NaN
 108370     17
 108371     16
 108372    NaN
 108373    NaN
 Name: VA_Left_Pressure, Length: 108374, dtype: object]